In [1]:
import pandas as pd
file_path = '/Users/yujinlee/Google_Drive/NYU/Fall_2023/Capstone/CrimeData'

In [2]:
boston = pd.read_csv(file_path + '/clean/boston_weekly_data_with_covariates.csv', low_memory = False)
seattle = pd.read_csv(file_path + '/clean/seattle_weekly_data_with_covariates.csv', low_memory = False)
nyc = pd.read_csv(file_path + '/clean/nyc_weekly_data.csv', low_memory = False)
la = pd.read_csv(file_path + '/clean/la_weekly_data.csv', low_memory = False)

In [4]:
nyc = nyc.rename(columns={"CASERATE": "case_count"})
la = la.rename(columns={"new_case": "case_count"})
boston = boston.rename(columns={"weekly_cases": "case_count"})

In [5]:
nyc['case_count'] = nyc['case_count'].fillna(0)

In [6]:
la = la.drop('caserate', axis=1)

In [7]:
nyc.dtypes

week                 object
ZCTA5CE10             int64
total_count         float64
total_population    float64
crime_rate          float64
e/pop_rate          float64
un_rate             float64
p_rate              float64
in_median           float64
in_mean             float64
case_count          float64
PCTPOS              float64
week_no               int64
y_hat               float64
dtype: object

In [28]:
nyc.head()

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat
0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,1,10.416994
1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,2,10.418465
2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,5,10.422879
3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,6,10.424351
4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,7,10.425822


In [29]:
boston.head()

,ZCTA5CE10,week,total_count,total_population,crime_rate,year,e/pop,un_rate,p_rate,in_median,in_mean,week_no,case_count
0,2108,2018-01-07,12,4218,28.449502,2018,61.2,8.3,10.8,136667.0,136667.0,1,0.0
1,2108,2018-01-14,16,4218,37.932670,2018,61.2,8.3,10.8,136667.0,136667.0,2,0.0
2,2108,2018-01-21,18,4218,42.674253,2018,61.2,8.3,10.8,136667.0,136667.0,3,0.0
3,2108,2018-01-28,16,4218,37.932670,2018,61.2,8.3,10.8,136667.0,136667.0,4,0.0
4,2108,2018-02-04,15,4218,35.561878,2018,61.2,8.3,10.8,136667.0,136667.0,5,0.0


In [11]:
la.dtypes

week                 object
ZCTA5CE10             int64
total_count         float64
total_population    float64
crime_rate          float64
e/pop_rate          float64
un_rate             float64
p_rate              float64
in_median             int64
in_mean               int64
case_count          float64
week_no               int64
y_hat               float64
dtype: object

In [8]:
nyc[nyc.isna().any(axis=1)]

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat


In [10]:
la[la.isna().any(axis=1)]

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,week_no,y_hat


In [12]:
boston['week'] = pd.to_datetime(boston['week'])
seattle['week'] = pd.to_datetime(seattle['week'])
nyc['week'] = pd.to_datetime(nyc['week'])
la['week'] = pd.to_datetime(la['week'])

# 1st Pre-trend Analysis

In [13]:
boston_1st = boston[(boston['week'] >= '2018-01-01') & (boston['week'] <= '2020-06-30')]
seattle_1st = seattle[(seattle['week'] >= '2018-01-01') & (seattle['week'] <= '2020-06-30')]
nyc_1st = nyc[(nyc['week'] >= '2018-01-01') & (nyc['week'] <= '2020-06-30')]
la_1st = la[(la['week'] >= '2018-01-01') & (la['week'] <= '2020-06-30')]

In [14]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
boston_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=boston_1st).fit()
boston_1st['y_hat'] = boston_model.predict(boston_1st)
print(boston_1st['y_hat'].mean())
print(boston_model.summary())

27.99919627873173
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     91.42
Date:                Sat, 02 Mar 2024   Prob (F-statistic):           2.18e-91
Time:                        23:54:47   Log-Likelihood:                -18310.
No. Observations:                4026   AIC:                         3.663e+04
Df Residuals:                    4020   BIC:                         3.667e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.5542      2.611  

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/1817823523.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  boston_1st['y_hat'] = boston_model.predict(boston_1st)


In [15]:
seattle_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=seattle_1st).fit()
seattle_1st['y_hat'] = seattle_model.predict(seattle_1st)
print(seattle_1st['y_hat'].mean())
print(seattle_model.summary())

17.361528338306698
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.269
Model:                            OLS   Adj. R-squared:                  0.268
Method:                 Least Squares   F-statistic:                     258.3
Date:                Sat, 02 Mar 2024   Prob (F-statistic):          1.43e-235
Time:                        23:54:52   Log-Likelihood:                -14183.
No. Observations:                3510   AIC:                         2.838e+04
Df Residuals:                    3504   BIC:                         2.841e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     27.8073      2.516 

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/3071156503.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seattle_1st['y_hat'] = seattle_model.predict(seattle_1st)


In [17]:
nyc_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=nyc_1st).fit()
nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
print(nyc_1st['y_hat'].mean())
print(nyc_model.summary())

10.129104589892655
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     1555.
Date:                Sat, 02 Mar 2024   Prob (F-statistic):               0.00
Time:                        23:55:10   Log-Likelihood:                -78258.
No. Observations:               23513   AIC:                         1.565e+05
Df Residuals:                   23508   BIC:                         1.566e+05
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.5270      0.279 

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/4163663363.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
/Users/yujinlee/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1918: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])


In [18]:
la_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=la_1st).fit()
la_1st['y_hat'] = la_model.predict(la_1st)
print(la_1st['y_hat'].mean())
print(la_model.summary())

9.48347537107679
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.217
Model:                            OLS   Adj. R-squared:                  0.216
Method:                 Least Squares   F-statistic:                     961.7
Date:                Sat, 02 Mar 2024   Prob (F-statistic):               0.00
Time:                        23:55:13   Log-Likelihood:                -68063.
No. Observations:               17401   AIC:                         1.361e+05
Df Residuals:                   17395   BIC:                         1.362e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.6532      0.718   

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/2167795371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_1st['y_hat'] = la_model.predict(la_1st)


In [16]:
# nyc_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + p_rate + un_rate + in_median + case_count', data=nyc_1st).fit()
# nyc_1st['y_hat'] = nyc_model.predict(nyc_1st)
# print(nyc_1st['y_hat'].mean())
# print(nyc_model.summary())

In [19]:
nyc_1st['city'] = 'nyc'
la_1st['city'] = 'la'
boston_1st['city'] = 'boston'
seattle_1st['city'] = 'seattle'

/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/765401552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nyc_1st['city'] = 'nyc'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/765401552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  la_1st['city'] = 'la'
/var/folders/yw/mp5v5fdx1_n4b01714b8bdl80000gn/T/ipykernel_58013/765401552.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [25]:
nycla_1st = pd.concat([nyc_1st, la_1st], axis=0)
nycboston_1st = pd.concat([nyc_1st, boston_1st], axis=0)
nycseattle_1st = pd.concat([nyc_1st, seattle_1st], axis=0)

In [26]:
nycla_1st

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat,city
0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,1,10.737083,nyc
1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,2,10.724234,nyc
2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,5,10.685688,nyc
3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,6,10.672839,nyc
4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,7,10.659990,nyc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37534,2020-06-07,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,8721.0,NaN,127,4.503287,la
37535,2020-06-14,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,9920.0,NaN,128,4.473889,la
37536,2020-06-21,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,14124.0,NaN,129,4.403950,la
37537,2020-06-28,91801,0.0,53320.0,0.000000,59.1,5.2,14.3,55832.0,77641.0,17174.0,NaN,130,4.349580,la


In [37]:
nycla_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + week_no+ p_rate + un_rate + in_median + case_count', data=nycla_1st).fit()
nycla_1st['y_hat_nycla'] = nycla_model.predict(nycla_1st)
print(nycla_1st['y_hat_nycla'].mean())
print(nycla_model.summary())

9.854514130976634
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.182
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     1302.
Date:                Sun, 03 Mar 2024   Prob (F-statistic):               0.00
Time:                        00:08:02   Log-Likelihood:            -1.5057e+05
No. Observations:               40914   AIC:                         3.012e+05
Df Residuals:                   40906   BIC:                         3.012e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept 

In [27]:
nycboston_1st.head()

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat,city,year,e/pop
0,2018-01-07,10001,99.0,27004.0,36.661235,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,1,10.737083,nyc,NaN,NaN
1,2018-01-14,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,2,10.724234,nyc,NaN,NaN
2,2018-02-04,10001,97.0,27004.0,35.920604,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,5,10.685688,nyc,NaN,NaN
3,2018-02-11,10001,104.0,27004.0,38.512813,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,6,10.672839,nyc,NaN,NaN
4,2018-02-18,10001,106.0,27004.0,39.253444,67.8,4.5,13.3,88526.0,151628.0,0.0,0.0,7,10.659990,nyc,NaN,NaN


In [36]:
nycboston_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + week_no + p_rate + un_rate + in_median + case_count', data=nycboston_1st).fit()
nycboston_1st['y_hat_nycboston'] = nycboston_model.predict(nycboston_1st)
print(nycboston_1st['y_hat_nycboston'].mean())
print(nycboston_model.summary())

12.741581046699723
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.318
Method:                 Least Squares   F-statistic:                     1833.
Date:                Sun, 03 Mar 2024   Prob (F-statistic):               0.00
Time:                        00:07:42   Log-Likelihood:            -1.0485e+05
No. Observations:               27539   AIC:                         2.097e+05
Df Residuals:                   27531   BIC:                         2.098e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept

In [35]:
nycseattle_model = smf.ols('crime_rate ~ C(city) + week_no:C(city) + week_no + p_rate + un_rate + in_median + case_count', data=nycseattle_1st).fit()
nycseattle_1st['y_hat_nycseattle'] = nycseattle_model.predict(nycseattle_1st)
print(nycseattle_1st['y_hat_nycseattle'].mean())
print(nycseattle_model.summary())

11.068519435290805
                            OLS Regression Results                            
Dep. Variable:             crime_rate   R-squared:                       0.210
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     1025.
Date:                Sun, 03 Mar 2024   Prob (F-statistic):               0.00
Time:                        00:05:49   Log-Likelihood:                -95913.
No. Observations:               27023   AIC:                         1.918e+05
Df Residuals:                   27015   BIC:                         1.919e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
I

In [34]:
nycseattle_1st.tail()

,week,ZCTA5CE10,total_count,total_population,crime_rate,e/pop_rate,un_rate,p_rate,in_median,in_mean,case_count,PCTPOS,week_no,y_hat,city,year,e/pop,y_hat_nycseattle
7482,2020-05-31,98199,30.0,22359.0,13.417416,NaN,3.4,5.5,108328.0,108328.0,0.0,NaN,126,10.297070,seattle,2020.0,69.0,16.348088
7483,2020-06-07,98199,16.0,22359.0,7.155955,NaN,3.4,5.5,108328.0,108328.0,2.0,NaN,127,10.024834,seattle,2020.0,69.0,16.171781
7484,2020-06-14,98199,13.0,22359.0,5.814214,NaN,3.4,5.5,108328.0,108328.0,8.0,NaN,128,9.195332,seattle,2020.0,69.0,15.634682
7485,2020-06-21,98199,21.0,22359.0,9.392191,NaN,3.4,5.5,108328.0,108328.0,7.0,NaN,129,9.341046,seattle,2020.0,69.0,15.728970
7486,2020-06-28,98199,19.0,22359.0,8.497697,NaN,3.4,5.5,108328.0,108328.0,1.0,NaN,130,10.183342,seattle,2020.0,69.0,16.274248


# 2nd Pre-trend Analysis

In [ ]:
boston_2nd = boston[(boston['week'] >= '2020-07-01') & (boston['week'] <= '2022-04-01')]
seattle_2nd = seattle[(seattle['week'] >= '2020-07-01') & (seattle['week'] <= '2022-04-01')]
nyc_2nd = nyc[(nyc['week'] >= '2020-07-01') & (nyc['week'] <= '2022-04-01')]
la_2nd = la[(la['week'] >= '2020-07-01') & (la['week'] <= '2022-04-01')]

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
boston_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + weekly_cases', data=boston_2nd).fit()
boston_2nd['y_hat'] = boston_model.predict(boston_2nd)
print(boston_2nd['y_hat'].mean())
print(boston_model.summary())

In [ ]:
seattle_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=seattle_2nd).fit()
seattle_2nd['y_hat'] = seattle_model.predict(seattle_2nd)
print(seattle_2nd['y_hat'].mean())
print(seattle_model.summary())

In [ ]:
nyc_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=nyc_2nd).fit()
nyc_2nd['y_hat'] = nyc_model.predict(nyc_2nd)
print(nyc_2nd['y_hat'].mean())
print(nyc_model.summary())

In [ ]:
la_model = smf.ols('crime_rate ~ week_no + p_rate + un_rate + in_median + case_count', data=la_2nd).fit()
la_2nd['y_hat'] = la_model.predict(la_2nd)
print(la_2nd['y_hat'].mean())
print(la_model.summary())